In [1]:
import pandas as pd
import os
from pyexeggutor import read_list
from metabolic_niche_space.utils import fast_groupby


/home/ec2-user/SageMaker/environments/mns/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%%time
# Data
output_directory="../data/training"
df_quality = pd.read_csv("../data/quality.tsv.gz", sep="\t", index_col=0)
quality_label="completeness_gte90.contamination_lt5"
# quality_label="completeness_gte50.contamination_lt10"

genome_to_clusterani = pd.read_csv(os.path.join(output_directory, quality_label, "genome_to_ani-cluster.tsv.gz"), sep="\t", index_col=0).iloc[:,0].astype("category")
X_genomic_traits = pd.read_csv(os.path.join(output_directory, "global.genomic_traits.kofam.bool-int.pathway_subset.tsv.gz"), sep="\t", index_col=0).astype(bool)
X_genomic_traits = X_genomic_traits.loc[X_genomic_traits.sum(axis=1)[lambda x: x > 0].index]
genomes = sorted(set(genome_to_clusterani.index) & set(X_genomic_traits.index))
X_genomic_traits = X_genomic_traits.loc[genomes]
genome_to_clusterani = genome_to_clusterani.loc[genomes]
# X_genomic_traits_clusterani_prevalence_ratio = fast_groupby(X_genomic_traits, genome_to_clusterani, method="mean")
X_genomic_traits_clusterani = fast_groupby(X_genomic_traits, genome_to_clusterani, method="sum").astype(bool)


eukaryotes = read_list(f"../data/cluster/ani/eukaryotic/{quality_label}/organisms.list", set)
prokaryotes = read_list(f"../data/cluster/ani/prokaryotic/{quality_label}/organisms.list", set)


# CPU times: user 11.6 s, sys: 1.22 s, total: 12.8 s
# Wall time: 12.8 s

Grouping rows by: 100%|██████████| 2124/2124 [00:00<00:00, 11274.30 column/s]


CPU times: user 10.1 s, sys: 604 ms, total: 10.7 s
Wall time: 10.7 s


In [3]:
X_genomic_traits.astype(int).to_csv(f"../data/training/{quality_label}/X.tsv.gz", sep="\t")
X_genomic_traits_clusterani.astype(int).to_csv(f"../data/training/{quality_label}/X_grouped.tsv.gz", sep="\t")
genome_to_clusterani.to_frame().to_csv(f"../data/training/{quality_label}/y.tsv.gz", sep="\t", header=None)

In [6]:
X_genomic_traits_clusterani.values.ravel().mean()

0.1629830127286115